In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TabNet: Attentive Interpretable Tabular Learning

https://arxiv.org/abs/1908.07442

The GCP platform provides a new buitin algorithm called TabNet. The training and deployment can be found at https://cloud.google.com/ai-platform/training/docs/algorithms?hl=en

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ml-on-gcp/blob/master/tutorials/explanations/ai-explanations-tabnet-algorithm.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ml-on-gcp/tree/master/tutorials/explanations/ai-explanations-tabnet-algorithm.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

This tutorial provides the sample code to visualize the explanation of TabNet algorithm with Synthetic 2 data output.

### Objective

The goal is to provide a sample plotting tool to visualize the output of TabNet which is helpful in explain the algorithm.

## Before you begin

Make sure you're running this notebook in a **GPU runtime** if you have that option. In Colab, select **Runtime** --> **Change runtime type**


This tutorial assumes you are running the notebook either in **Colab** or **Cloud AI Platform Notebooks**.

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project.](https://console.cloud.google.com/cloud-resource-manager)

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AI Platform Training & Prediction and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

In [ ]:
PROJECT_ID = "[<your-project-id>]"

### Authenticate your GCP account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

In [ ]:
import os
import sys
import warnings

warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# If you are running this notebook in Colab, follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.


def install_dlvm_packages():
    !pip install tabulate


if 'google.colab' in sys.modules:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()
    !pip install witwidget --quiet
    !pip install tensorflow==1.15.0 --quiet
    !gcloud config set project $PROJECT_ID
    
    
elif "DL_PATH" in os.environ:
    install_dlvm_packages()

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you submit a training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. AI Platform runs
the code from this package. In this tutorial, AI Platform also saves the
trained model that results from your job in the same bucket. You can then
create an AI Platform model version based on this output in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets. 

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Cloud
AI Platform services are
available](https://cloud.google.com/ml-engine/docs/tensorflow/regions). You may
not use a Multi-Regional Storage bucket for training with AI Platform.

In [ ]:
BUCKET_NAME = "[<your-bucket-name>]"
REGION = "us-central1"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
!gsutil mb -l $REGION gs://$BUCKET_NAME

### Import libraries

Import the libraries we'll be using in this tutorial. 

In [ ]:
import numpy as np
import json
from google.cloud import storage
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

## Read the Tabnet's output of Syn2 data

After training and serving the algorithm, you put the output in a GCS file.

There is a sample output on Synthetic data: 
gs://cloud-samples-data/ai-platform/synthetic/tab_net_output/syn2

You can copy this output to your bucket for testing. Running prediction on your own data will generate the output with similar format.


In [ ]:
BUCKET_NAME = "<your-bucket-name>"
PREDICTION_FILE = "your-prediction-file"
MASK_KEY = "aggregated_mask_values"

HEADER = [("feat_" + str(i)) for i in range(1, 12)]
HEADER

### Download the contents of the blob as a string and then parse it using json.loads() method

In [ ]:
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)
blob = bucket.blob(PREDICTION_FILE)
f = blob.download_as_string(client=None).decode("utf-8").strip()
predictions = f.split("\n")

### Concatenate the mask values
the output is a matrix having Nxk (N is the number of outputs, k is the size of each mask)

In [ ]:
masks = []
for prediction in predictions:
    prediction = json.loads(prediction)
    masks.append(prediction[MASK_KEY])
masks = np.matrix(masks)
masks.shape

## Visualize the matrix. The lighter color indicates more important feature.

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(121)
ax.imshow(masks[:50, :], interpolation='bilinear', cmap=cm.Greys_r)
ax.set_xlabel('Features')
ax.set_ylabel('Sample index')
ax.xaxis.set_ticks(np.arange(len(HEADER)))
ax.set_xticklabels(HEADER, rotation='vertical')
plt.show()

If your Cloud Storage bucket doesn't contain any other objects and you would like to delete it, run `gsutil rm -r gs://$BUCKET_NAME`.

## What's next?

To learn more about TabNet, check out the resources here.

* [TabNet: Attentive Interpretable Tabular Learning](https://arxiv.org/abs/1908.07442)
* [https://arxiv.org/abs/1908.07442](https://cloud.google.com/ai-platform/training/docs/algorithms?hl=en) 